In [1]:
from pymongo import MongoClient
import pandas as pd
from pandas.io.json import json_normalize

pd.set_option('max_columns', 50)

In [2]:
# conexión con  mongo
client = MongoClient("mongodb://localhost:27017/")

# conexión con la base de datos "db_companies"
db = client.db_companies

#coleción companies
collection_companies = db.companies

# query pidiendo TODO
query = collection_companies.find()

In [3]:
# convertimos a pandas.DataFrame
df = pd.DataFrame(query)

# vemos el resultado
df.head()

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,deadpooled_year,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,image,products,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,acquisition,acquisitions,offices,milestones,video_embeds,screenshots,external_links,partners,founded_month,founded_day,deadpooled_month,deadpooled_day,deadpooled_url,ipo
0,52cdef7c4bab8bd675297d8b,AdventNet,abc3,http://www.crunchbase.com/company/adventnet,http://adventnet.com,,,manageengine,enterprise,600.0,1996.0,2.0,,Zoho ManageEngine,pr@adventnet.com,925-924-9500,Server Management Software,2007-05-25 19:24:22,Wed Oct 31 18:26:09 UTC 2012,"<p>AdventNet is now <a href=""/company/zoho-man...","{'available_sizes': [[[150, 55], 'assets/image...",[],"[{'is_past': True, 'title': 'CEO and Co-Founde...",[],"[{'title': 'DHFH', 'is_past': True, 'provider'...",$0,[],[],None,[],"[{'description': 'Headquarters', 'address1': '...",[],[],"[{'available_sizes': [[[150, 94], 'assets/imag...",[],[],NaN,NaN,NaN,NaN,NaN,NaN
1,52cdef7c4bab8bd675297d8a,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,2005.0,1.0,"wiki, seattle, elowitz, media-industry, media-...",,info@wetpaint.com,206.859.6300,Technology Platform Company,2007-05-25 06:51:27,Sun Dec 08 07:15:44 UTC 2013,<p>Wetpaint is a technology platform company t...,"{'available_sizes': [[[150, 75], 'assets/image...","[{'name': 'Wikison Wetpaint', 'permalink': 'we...","[{'is_past': False, 'title': 'Co-Founder and V...","[{'competitor': {'name': 'Wikia', 'permalink':...",[],$39.8M,"[{'id': 888, 'round_code': 'a', 'source_url': ...",[],"{'price_amount': 30000000, 'price_currency_cod...",[],"[{'description': '', 'address1': '710 - 2nd Av...","[{'id': 5869, 'description': 'Wetpaint named i...",[],"[{'available_sizes': [[[150, 86], 'assets/imag...",[{'external_url': 'http://www.geekwire.com/201...,[],10.0,17.0,NaN,NaN,NaN,NaN
2,52cdef7c4bab8bd675297d8c,Zoho,abc4,http://www.crunchbase.com/company/zoho,http://zoho.com,http://blogs.zoho.com/,http://blogs.zoho.com/feed,zoho,software,1600.0,2005.0,3.0,"zoho, officesuite, spreadsheet, writer, projec...",,info@zohocorp.com,1-888-204-3539,Online Business Apps Suite,Fri May 25 19:30:28 UTC 2007,Wed Oct 30 00:07:05 UTC 2013,"<p>Zoho offers a suite of Business, Collaborat...","{'available_sizes': [[[150, 55], 'assets/image...","[{'name': 'Zoho Office Suite', 'permalink': 'z...","[{'is_past': False, 'title': 'CEO and Founder'...","[{'competitor': {'name': 'Empressr', 'permalin...",[],$0,[],[],None,[],"[{'description': 'Headquarters', 'address1': '...","[{'id': 388, 'description': 'Zoho Reaches 2 Mi...","[{'embed_code': '<object width=""430"" height=""2...",[],[{'external_url': 'http://www.online-tech-tips...,[],9.0,15.0,NaN,NaN,NaN,NaN
3,52cdef7c4bab8bd675297d8f,Omnidrive,omnidrive,http://www.crunchbase.com/company/omnidrive,http://www.omnidrive.com,http://www.omnidrive.com/blog,http://feeds.feedburner.com/omnidrive,Nomadesk,network_hosting,NaN,2005.0,2008.0,"storage, sharing, edit, online",None,info@omnidrive.com,660-675-5052,None,Sun May 27 03:25:32 UTC 2007,Tue Jul 02 22:48:04 UTC 2013,"<p>Currently in public beta, Omnidrive makes i...","{'available_sizes': [[[150, 85], 'assets/image...","[{'name': 'Omnidrive', 'permalink': 'omnidrive'}]","[{'is_past': True, 'title': 'Co-founder', 'per...","[{'competitor': {'name': 'Dropbox', 'permalink...",[],$800k,"[{'id': 225, 'round_code': 'angel', 'source_ur...",[],None,[],"[{'description': '', 'address1': 'Suite 200', ...",[],[],[],[],[],11.0,1.0,9.0,15.0,,None
4,52cdef7c4bab8bd675297d8d,Digg,digg,http://www.crunchbase.com/company/digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60.0,2004.0,NaN,"community, social, news, bookmark

In [4]:
# Todo el dataset
df.shape

(18801, 42)

### Filtro seleccionado: 

Filtramos las empresas que cumplen: son de videjuegos, más de 500 empleados, abiertas antes de 2012, y no en USA por problemas políticos


In [5]:
companies = collection_companies.find({"$and":[
                                            {"offices": {"$exists": True}},
                                            {"offices": {"$ne": None}}, 
                                            {"category_code": {"$exists": True}},
                                            {"category_code": {"$ne": None}}, 
                                            {"founded_year": {"$exists": True}}, 
                                            {"founded_year": {"$gte": 2003}},
                                            {"deadpooled_year": None},
                                            {"number_of_employees": {"$exists": True}},
                                            {"number_of_employees": {"$gte": 10}},
                                            {"total_money_raised": {"$exists": True}},
                                            {"total_money_raised":{"$ne":None}},
                                            {"total_money_raised": {"$not":{"$size":0}}}, 
                                            {"$or": [
                                                {"total_money_raised": {"$gte": 1_000_000}},
                                                {"category_code": "design" } ,
                                                {"category_code": "web" } , 
                                                {"category_code": "software" } , 
                                                {"category_code": "games_video" } , 
                                                {"category_code": "mobile" } , 
                                                {"category_code": "enterprise" } ,   
                                                {"category_code": "analytics" } ,
                                                {'category_code': "search"},
                                                {'category_code': "network_hosting"} ,   
                                                {"category_code": "photo_video" } ,   
                                                ]} ,

                                            ]
                                            },

                                            # descartamos elementos que no interesan
                                            {"_id": 0, "crunchbase_url": 0, "products": 0, 
                                             "acquisition": 0, "acquisitions": 0, "video_embeds": 0, 
                                             "screenshots": 0, "external_links": 0, "partners": 0, 
                                             "image": 0, 
                                             'deadpooled_day': 0, 'deadpooled_month': 0, 
                                             'deadpooled_url': 0, 'deadpooled_year': 0,
                                            }
                                     )


In [6]:

lst = list(companies)
len(lst)

1683

In [7]:
df = pd.DataFrame(lst)
df.head(2)

,name,permalink,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,offices,milestones,ipo
0,Geni,geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18,2006,6.0,1.0,"geni, geneology, social, family, genealogy",,,,Geneology social network site,Thu May 31 19:52:34 UTC 2007,Wed Oct 10 14:01:29 UTC 2012,<p>Geni is an online community of casual and e...,"[{'is_past': False, 'title': 'CEO', 'person': ...","[{'competitor': {'name': 'Ancestry', 'permalin...",[],$16.5M,"[{'id': 6, 'round_code': 'a', 'source_url': ''...",[],"[{'description': 'Headquarters', 'address1': '...","[{'id': 15460, 'description': 'Announced hire ...",None
1,Powerset,powerset,http://powerset.com,http://blog.powerset.com/,http://blog.powerset.com/atom.xml,Powerset,search,60,2006,10.0,NaN,"searchengine, google, techcrunch40, natural-la...",,info@powerset.com,415-848-7000,,Mon Jun 11 19:33:13 UTC 2007,Sat Mar 16 06:13:05 UTC 2013,<p>Powerset is a search engine focused on natu...,"[{'is_past': False, 'title': 'Founder / Produc...","[{'competitor': {'name': 'Hakia', 'permalink':...","[{'title': '', 'is_past': False, 'provider': {...",$22.5M,"[{'id': 22, 'round_code': 'a', 'source_url': '...",[],"[{'description': None, 'address1': '475 Branna...",[],None


In [8]:
columns_to_count = ['competitions', 'funding_rounds', 'investments' ,'providerships', 
                    'relationships', 'milestones']

def count_elements(x): 
    try: 
        return len(x)
    except: 
        return 0

def clean_lists(dataframe, cols): 
    for column in cols: 
        dataframe[column] = dataframe[column].apply(count_elements)
    return dataframe

df = clean_lists(df, columns_to_count)
df.head()

,name,permalink,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,offices,milestones,ipo
0,Geni,geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18,2006,6.0,1.0,"geni, geneology, social, family, genealogy",,,,Geneology social network site,Thu May 31 19:52:34 UTC 2007,Wed Oct 10 14:01:29 UTC 2012,<p>Geni is an online community of casual and e...,13,2,0,$16.5M,3,0,"[{'description': 'Headquarters', 'address1': '...",1,None
1,Powerset,powerset,http://powerset.com,http://blog.powerset.com/,http://blog.powerset.com/atom.xml,Powerset,search,60,2006,10.0,NaN,"searchengine, google, techcrunch40, natural-la...",,info@powerset.com,415-848-7000,,Mon Jun 11 19:33:13 UTC 2007,Sat Mar 16 06:13:05 UTC 2013,<p>Powerset is a search engine focused on natu...,20,13,1,$22.5M,3,0,"[{'description': None, 'address1': '475 Branna...",0,None
2,Mahalo,mahalo,http://mahalo.com,http://blog.mahalo.com/,http://blog.mahalo.com/feed/,MahaloDotCom,web,40,2007,3.0,1.0,"search, search-engine, human-powered-search, aaa",,jason@mahalo.com,,Life long learning content,Thu Jun 14 03:42:20 UTC 2007,Fri May 17 04:34:19 UTC 2013,<p>Mahalo is a human powered search engine fou...,13,9,0,$21M,2,0,"[{'description': '', 'address1': '3525 Eastham...",0,None
3,Kyte,kyte,http://www.kyte.com,http://kyte.com/blog,http://feeds.feedburner.com/kyte_blog,kyte,games_video,40,2006,12.0,1.0,"video, mobile, iphone-app, video-platform, mob...",,info@kyte.com,,Online & Mobile Video Platform,Thu Jun 14 18:26:11 UTC 2007,Mon Oct 28 09:34:37 UTC 2013,<p>Kyte is the online and mobile video platfor...,11,10,0,$23.4M,4,0,"[{'description': None, 'address1': '442 Post S...",0,None
4,Jingle Networks,jingle-networks,http://jinglenetworks.com,,,Marchex,mobile,35,2005,9.0,1.0,"directoryassistance, advertising, mobile",,pr@jinglenetworks.com,212-481-4114 x237,Voice and Mobile Search,Sat Jun 16 05:53:38 UTC 2007,Mon Jun 03 05:26:22 UTC 2013,<p>Jingle Networks is the leading provider of ...,11,0,2,$88.7M,7,0,"[{'description': '', 'address1': '475 Park Ave...",2,None


In [9]:
# offices

# df.loc[8, 'offices']

df.loc[8, 'offices']

[{'description': 'San Francisco Office',
  'address1': '410 Townsend',
  'address2': 'Suite 400',
  'zip_code': '94107',
  'city': 'San Francisco',
  'state_code': 'CA',
  'country_code': 'USA',
  'latitude': 37.392936,
  'longitude': -122.07948},
 {'description': 'Los Angeles Office',
  'address1': '6725 W Sunset Blvd',
  'address2': 'Suite 320',
  'zip_code': '90028',
  'city': 'Los Angeles',
  'state_code': 'CA',
  'country_code': 'USA',
  'latitude': None,
  'longitude': None},
 {'description': 'Budapest Office',
  'address1': '',
  'address2': '',
  'zip_code': '',
  'city': 'Budapest',
  'state_code': None,
  'country_code': 'HUN',
  'latitude': None,
  'longitude': None}]

In [31]:
dfx = pd.DataFrame({'A':['a','b'],
               'B':[['A1', 'A2'],['A1', 'A2', 'A3']]})
dfx

,A,B
0,a,"[A1, A2]"
1,b,"[A1, A2, A3]"


In [32]:
dfx['B'].apply(lambda x: pd.Series(x)).stack().reset_index(level=1, drop=True).to_frame('B').join(dfx[['A']], how='left')

,B,A
0,A1,a
0,A2,a
1,A1,b
1,A2,b
1,A3,b


In [28]:
cols = [col for col in df.columns if col != 'offices']
cols

['name',
 'permalink',
 'homepage_url',
 'blog_url',
 'blog_feed_url',
 'twitter_username',
 'category_code',
 'number_of_employees',
 'founded_year',
 'founded_month',
 'founded_day',
 'tag_list',
 'alias_list',
 'email_address',
 'phone_number',
 'description',
 'created_at',
 'updated_at',
 'overview',
 'relationships',
 'competitions',
 'providerships',
 'total_money_raised',
 'funding_rounds',
 'investments',
 'milestones',
 'ipo',
 'coin',
 'total_amount_raised']

In [30]:
cols = [col for col in df.columns if col != 'offices']
df['offices'].apply(lambda x: pd.Series(x)).stack().reset_index(level=1, drop=True).to_frame('offices').join(df[cols], how='left')

,offices,name,permalink,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,milestones,ipo,coin,total_amount_raised
0,"{'description': 'Headquarters', 'address1': '9...",Geni,geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18,2006,6.0,1.0,"geni, geneology, social, family, genealogy",,,,Geneology social network site,Thu May 31 19:52:34 UTC 2007,Wed Oct 10 14:01:29 UTC 2012,<p>Geni is an online community of casual and e...,13,2,0,$16.5M,3,0,1,None,DOL,16500000.0
1,"{'description': None, 'address1': '475 Brannan...",Powerset,powerset,http://powerset.com,http://blog.powerset.com/,http://blog.powerset.com/atom.xml,Powerset,search,60,2006,10.0,NaN,"searchengine, google, techcrunch40, natural-la...",,info@powerset.com,415-848-7000,,Mon Jun 11 19:33:13 UTC 2007,Sat Mar 16 06:13:05 UTC 2013,<p>Powerset is a search engine focused on natu...,20,13,1,$22.5M,3,0,0,None,DOL,22500000.0
2,"{'description': '', 'address1': '3525 Eastham ...",Mahalo,mahalo,http://mahalo.com,http://blog.mahalo.com/,http://blog.mahalo.com/feed/,MahaloDotCom,web,40,2007,3.0,1.0,"search, search-engine, human-powered-search, aaa",,jason@mahalo.com,,Life long learning content,Thu Jun 14 03:42:20 UTC 2007,Fri May 17 04:34:19 UTC 2013,<p>Mahalo is a human powered search engine fou...,13,9,0,$21M,2,0,0,None,DOL,21000000.0
3,"{'description': None, 'address1': '442 Post St...",Kyte,kyte,http://www.kyte.com,http://kyte.com/blog,http://feeds.feedburner.com/kyte_blog,kyte,games_video,40,2006,12.0,1.0,"video, mobile, iphone-app, video-platform, mob...",,info@kyte.com,,Online & Mobile Video Platform,Thu Jun 14 18:26:11 UTC 2007,Mon Oct 28 09:34:37 UTC 2013,<p>Kyte is the online and mobile video platfor...,11,10,0,$23.4M,4,0,0,None,DOL,23400000.0
4,"{'description': '', 'address1': '475 Park Ave ...",Jingle Networks,jingle-networks,http://jinglenetworks.com,,,Marchex,mobile,35,2005,9.0,1.0,"directoryassistance, advertising, mobile",,pr@jinglenetworks.com,212-481-4114 x237,Voice and Mobile Search,Sat Jun 16 05:53:38 UTC 2007,Mon Jun 03 05:26:22 UTC 2013,<p>Jingle Networks is the leading provider of ...,11,0,2,$88.7M,7,0,2,None,DOL,88700000.0
5,"{'description': 'Jajah HQ', 'address1': '2513 ...",Jajah,jajah,http://jajah.com,http://blog.jajah.com,http://blog.jajah.com/index.php?/feeds/index.rss2,jajah,mobile,110,2005,1.0,1.0,"voip, jajah, jahjah, jaha, jaja, telephony, mo...",,press@jajah.com,+1-650-963-4847,IP Communications Platform,Mon Jul 02 19:53:59 UTC 2007,Thu May 09 17:14:48 UTC 2013,<p>Jajah is a VoIP service that gives you lowe...,19,12,0,$33M,4,0,4,None,DOL,33000000.0
6,"{'description': 'Livestream HQ', 'address1': '...",Livestream,livestream,http://www.livestream.com,http://www.livestream.com/blog/,http://www.livestream.com/blog/?feed=rss2,livestream,games_video,120,2007,5.0,1.0,"mogulus, ustream, justin-tv, live-broadcast, s...",Mogulus,info@livestream.com,(646) 495 9707,Live streaming platform and website,Sat Jun 23 05:37:56 UTC 2007,Tue Sep 17 12:19:38 UTC 2013,<p>Livestream&#8217;s mission is to connect pe...,24,12,0,$14.7M,4,0,0,None,DOL,14700000.0
7,"{'description': '', 'address1': '131 Varick St...",AdaptiveBlue,adaptiveblue,http://www.getglue.com,http://blog.adaptiveblue.com,http://blog.adaptiveblue.com/?feed=rss2,GetGlue,games_video,15,2007,NaN,NaN,"semantic-web, contextual-browsing, recommendat...",None,support@getglue.com,,,Tue Feb 05 22:28:14 UTC 2008,Wed Jul 18 06:26:47 UTC 2012,<p>AdaptiveBlue is the company that develops G...,7,3,0,$24M,4,0,0,None,DOL,24000000.0
8,"{'description': 'San Francisco Office', 'addre...",Ustream,ustream,http://www.ustream.tv,http://www.ustream.com/blog,http://www.ustream.com/blog/feed,USTREAM,games_video,250,2007,NaN,NaN,"live, stream, streaming

In [12]:
# Coins

COINS = {
        '$': 'DOL', 
        '€': 'EUR', 
        'C$': 'CAD', 
        'C': 'CAD', 
        '£': 'GBP', 
        '¥': 'JPY', 
        'kr': 'SEK'
    }

def get_coin(x, coins=COINS): 
    
    for coin, name in coins.items(): 
        if coin in x: 
            return name
    else: 
        return 'unknown'
        
df['coin'] = df.total_money_raised.apply(get_coin)
df.coin.value_counts()

DOL    1625
EUR      43
GBP      12
JPY       2
SEK       1
Name: coin, dtype: int64

In [13]:
MULTIPLES = {'B': 1e9, 'M': 1e6, 'k': 1e3}
def get_value(x, coins=COINS, multiples=MULTIPLES): 
    try: 
        # delete coin simbol
        for k in coins.keys(): 
            x = x.replace(k, "")

        # multiply if necesary. 
        for item, number in multiples.items(): 
            if item in x: 
                x = x.replace(item, "")
                return float(x) * number
        return float(x)
    except: 
        return x

get_value('$1.44M')

1440000.0

In [14]:
df['total_amount_raised'] = df.total_money_raised.apply(get_value)
df.head()

,name,permalink,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,offices,milestones,ipo,coin,total_amount_raised
0,Geni,geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18,2006,6.0,1.0,"geni, geneology, social, family, genealogy",,,,Geneology social network site,Thu May 31 19:52:34 UTC 2007,Wed Oct 10 14:01:29 UTC 2012,<p>Geni is an online community of casual and e...,13,2,0,$16.5M,3,0,"[{'description': 'Headquarters', 'address1': '...",1,None,DOL,16500000.0
1,Powerset,powerset,http://powerset.com,http://blog.powerset.com/,http://blog.powerset.com/atom.xml,Powerset,search,60,2006,10.0,NaN,"searchengine, google, techcrunch40, natural-la...",,info@powerset.com,415-848-7000,,Mon Jun 11 19:33:13 UTC 2007,Sat Mar 16 06:13:05 UTC 2013,<p>Powerset is a search engine focused on natu...,20,13,1,$22.5M,3,0,"[{'description': None, 'address1': '475 Branna...",0,None,DOL,22500000.0
2,Mahalo,mahalo,http://mahalo.com,http://blog.mahalo.com/,http://blog.mahalo.com/feed/,MahaloDotCom,web,40,2007,3.0,1.0,"search, search-engine, human-powered-search, aaa",,jason@mahalo.com,,Life long learning content,Thu Jun 14 03:42:20 UTC 2007,Fri May 17 04:34:19 UTC 2013,<p>Mahalo is a human powered search engine fou...,13,9,0,$21M,2,0,"[{'description': '', 'address1': '3525 Eastham...",0,None,DOL,21000000.0
3,Kyte,kyte,http://www.kyte.com,http://kyte.com/blog,http://feeds.feedburner.com/kyte_blog,kyte,games_video,40,2006,12.0,1.0,"video, mobile, iphone-app, video-platform, mob...",,info@kyte.com,,Online & Mobile Video Platform,Thu Jun 14 18:26:11 UTC 2007,Mon Oct 28 09:34:37 UTC 2013,<p>Kyte is the online and mobile video platfor...,11,10,0,$23.4M,4,0,"[{'description': None, 'address1': '442 Post S...",0,None,DOL,23400000.0
4,Jingle Networks,jingle-networks,http://jinglenetworks.com,,,Marchex,mobile,35,2005,9.0,1.0,"directoryassistance, advertising, mobile",,pr@jinglenetworks.com,212-481-4114 x237,Voice and Mobile Search,Sat Jun 16 05:53:38 UTC 2007,Mon Jun 03 05:26:22 UTC 2013,<p>Jingle Networks is the leading provider of ...,11,0,2,$88.7M,7,0,"[{'description': '', 'address1': '475 Park Ave...",2,None,DOL,88700000.0


In [15]:
df.dtypes

name                    object
permalink               object
homepage_url            object
blog_url                object
blog_feed_url           object
twitter_username        object
category_code           object
number_of_employees      int64
founded_year             int64
founded_month          float64
founded_day            float64
tag_list                object
alias_list              object
email_address           object
phone_number            object
description             object
created_at              object
updated_at              object
overview                object
relationships            int64
competitions             int64
providerships            int64
total_money_raised      object
funding_rounds           int64
investments              int64
offices                 object
milestones               int64
ipo                     object
coin                    object
total_amount_raised    float64
dtype: object

In [16]:
df.head(1)

,name,permalink,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,offices,milestones,ipo,coin,total_amount_raised
0,Geni,geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18,2006,6.0,1.0,"geni, geneology, social, family, genealogy",,,,Geneology social network site,Thu May 31 19:52:34 UTC 2007,Wed Oct 10 14:01:29 UTC 2012,<p>Geni is an online community of casual and e...,13,2,0,$16.5M,3,0,"[{'description': 'Headquarters', 'address1': '...",1,None,DOL,16500000.0


In [17]:
from datetime import datetime

def create_date(day, month, year): 
    print(day, month, year)
    if not day: 
        day = 1
    if not month: 
        month = 1
    day, month, year = int(day), int(month), int(year)
    date_str = '{}-{}-{}'.format(day, month, year)
    return datetime.strptime(date_str, '%d-%m-%Y')
    
    
create_date(None, 12.0, 2014)
    

None 12.0 2014


datetime.datetime(2014, 12, 1, 0, 0)

In [18]:
df['founded_date'] = df.apply(lambda x: create_date(x['founded_day'], x['founded_month'], x['founded_year']), 
                             axis=1)
df.head()

1.0 6.0 2006
nan 10.0 2006
1.0 6.0 2006
nan 10.0 2006


ValueError: ('cannot convert float NaN to integer', 'occurred at index 1')